# Introduction 

This notebook provides the minimum necessary code to access data from IMONNIT's API. For an overview of the architecture, see the readme.md file of the project where is notebook is hosted.

# 1. Required Input Parameters

## 1.1 Host

The list of services available via API can be found on [MONNIT website](https://www.imonnit.com/API). As noted there, **all timestamps are in UTC (Universal Coordinated Time) both for the input and for the output.**, and as such needs be converted.

MONNIT API services are divided in groups (Authentication, Lookup, Account, etc.), some of which allows an entire web application to emulate MONNIT's own end user website. This is beyond the scope of this notebook, which only seeks to acquire data. Before any of the API services can be used, an API key is required. This can be obtained from the **Authentication** group, by typing the username and password. 



The following end-point of interest can be found under the **DataMessage** group, as follows:

![monnit_api](monnit_api.png)


As shown in the image, **the data API requests must be performed for a maximum of a week time window.** Moreover, from the example:

`https://www.imonnit.com/xml/SensorDataMessages/Z3Vlc3Q6cGFzc3dvcmQ=?sensorID=101&fromDate=2011/01/01 6:22:14 PM&toDate=2011/01/02 6:22:14 PM`

we can obtain the host for our API requests:

In [1]:
host = 'https://www.imonnit.com/xml'

We can also see that an API request is formatted in the folling manner:

`host + api_key + sensorID + timewindow`

which we construct in the next sub-sections.

## 1.2 Authentication

As noted before, the API key must be obtained via the **Authentication** group in the website. Here, the file being loaded is a simple .txt file which contains said key on the first line, and nothing else.

In [2]:
with open('auth.txt', 'r') as f:
    api_key = f.readline().rstrip()

## 1.3 SensorID

To obtain the list of sensor IDs, we need to use another API service, from the **Sensor** group, which according to the website can take as input one of the following parameters:

```
Name:	String (optional)	Filters list to sensors with names containing this string. (case-insensitive)
NetworkID:	Integer (optional)	Filters list to sensor that belong to this network id
ApplicationID:	Integer (optional)	Filters list to sensor that are this application type
Status:	Integer (optional)	Filters list to sensor that match this status
```

**We can obtain the sensor name by observing it's details on iMonnit website**:

![locate_gatewayid](locate_gatewayid.png)



In [3]:
request = host+'/SensorList/'+api_key+'?name=Water Temperature - 513030-W-1 (mauka)'
import requests
request = requests.post(request)

In [4]:
print(request.text)

<?xml version="1.0" encoding="utf-8"?>
<SensorRestAPI xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema">
  <Method>SensorList</Method>
  <Result xsi:type="xsd:collection"><APISensorList>
  <APISensor SensorID="513030" ApplicationID="65" CSNetID="12648" SensorName="Water Temperature - 513030-W-1 (mauka)" LastCommunicationDate="11/25/2019 4:32:30 AM" NextCommunicationDate="11/25/2019 6:32:30 AM" LastDataMessageMessageGUID="be6b4d5d-3cfb-4f86-8edd-ed620e324705" PowerSourceID="1" Status="0" CanUpdate="True" CurrentReading="116.1° F" BatteryLevel="100" SignalStrength="78" AlertsActive="True" CheckDigit="BSVZ" AccountID="7124" MonnitApplicationID="65" />
</APISensorList></Result>
</SensorRestAPI>



However, depending on the subscription type, it may not be available. As such, **it is important that the Sensor ID is always documented on the sensor name, which was done as shown in the image:

In [9]:
sensor_id = '513030'

## 1.4 Time Window

Finally, we set the time window using the format to request data, discussed on this notebook Section 1.1 and make the request:

In [10]:
start_date = '2019/10/22 12:00:00 AM'
end_date = '2019/10/23 12:00:00 AM'

time_window = 'fromDate='+start_date+'&toDate='+end_date

In [11]:
request = host+'/SensorDataMessages/'+api_key+'?sensorID='+sensor_id+'&'+time_window
request = requests.post(request)

In [25]:
from xml.dom.minidom import parse, parseString

dom = parseString(request.text)
print(dom.toprettyxml(indent="\t"))

<?xml version="1.0" ?>
<SensorRestAPI xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
	
  
	<Method>SensorDataMessages</Method>
	
  
	<Result xsi:type="xsd:collection">
		<APIDataMessageList>
			
  
			<APIDataMessage Battery="100" Data="48" DataMessageGUID="22b9423d-0272-442d-8344-1e6e982bb05f" DataTypes="TemperatureData" DataValues="48" DisplayData="118.4° F" GatewayID="112709" MessageDate="10/23/2019 10:48:51 PM" MetNotificationRequirements="False" PlotLabels="Fahrenheit" PlotValue="118.4" PlotValues="118.4" SensorID="513030" SignalStrength="81" State="16" Voltage="3.06"/>
			
  
			<APIDataMessage Battery="100" Data="45.9" DataMessageGUID="c759ae7d-9831-4cf2-83f3-0150131246aa" DataTypes="TemperatureData" DataValues="45.9" DisplayData="114.6° F" GatewayID="112709" MessageDate="10/23/2019 8:49:32 PM" MetNotificationRequirements="False" PlotLabels="Fahrenheit" PlotValue="114.62" PlotValues="114.62" SensorID="513030" SignalStrength="8

Finally, we store the raw file to disk:

In [26]:
file_handle = open("imonnit_data.xml","w")
dom.writexml(file_handle)
file_handle.close()